# TITLE CELL

OVERALL, EXPLAIN CODE SNIPPETS, have them interact with the rest of the algorithms, and explain the overall approach.

## The environment

Description of the environment, the task, reward, the MARL setting...   


The problem illustrated in this notebook is a Multi-Agent Reinforcement Learning (MARL) consisting of 2 players playing one against the other at a game called Connect 4. Connect 4 is a two-player game. The goal is to get 4 tokens aligned horizontally, vertically or diagonaly. The token are placed turn by turn and fall vertically until the bottom or another token. If the grid is full and nobody won, the game is a draw. The grid is a rectangle 7 cells wide and 6 cells high. The observation space is a 6*7 matrix filled with 1 if the agent has token on this cell and 0 if either nobody or the other agent has a token on this cell. The action space is an interger between 0 and 6 included that indicates on each column to drop a token.  

The reward is +1 for a win, 0 for a draw and -1 for a lose or for an illegal move. 




## First Ingredient: Tree Search

### Deterministic Tree Search: from BFS to Alpha-Beta Pruning


Given a certain state of the grid, we can compute all the possible positions with each action we can take. Then we can do the same for each action of the oppenent. This way, we can build a tree with each node representing the state of the board and the edges an action taken. A naive solution would be to explore all the tree until a certain depth. This algorithm thus works by taking the This method called Breadth-First Search (BFS) is simple to implement but is both memory and computation expensive as the number of nodes is multiplied by 7 at each depth. If we compute a single combination of moves from the node until the end, we obtain the Depth-First Search (DFS). In this case, to compute the value at each node we proceed according to the following way : if it is a terminal node, we assign a value of & in case of a win, 0 for a draw and -1 for a less. For non terminal nodes, if it is the agent's turn we take the maximum value among its children. If it is the oppenent's turn, we take the minimum value. It requires less memory compared to BFS but it may waste a lot of time exploring unpromising branches. A way to solve this problem is a method called Alpha-Beta Pruning. 


### Monte-Carlo Tree Search

Describe MCTS, link it to UCB, 

Adding heuristics to MCTS

Mention decision time planning

## Second Ingredient: Function Approximation

### Use of a value network and policy network in MCTS: the first step in AlphaGo

Show that a poorly trained value network/policy network are improved by MCTS :
- show that value estimates are improved by MCTS
- show that policy estimates are improved by MCTS (compared to depth 0)

### How to train these networks (for the first step in AlphaGo)

## Third Ingredient: Self-Play

### How to generate training data

### How to train the networks

Mention the details and how they are related to the different algorithms we have seen during the course (is there a replay buffer, is it on policy or off policy, etc.)

Mention background planning

Mention (well, it's in it) policy gradients.--> show what happens when you do MCTS with the trained policy but at depth zero (i.e. without any search and only using the policy) and with a very shallow search...show the importance of search.

## Result! AlphaZero

### Describe AlphaZero
decition time planning with MCTS, background planning with self-play, and the use of deep neural networks.

Show a complete training

### Variants (MuZero, Gumbel MuZero, other like Efficient Zero?)